In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
df1=pd.read_csv('/content/drive/MyDrive/DonaldTrump (np) 2009-2016 - DonaldTrumpTweets.csv')
df2=pd.read_csv("/content/drive/MyDrive/trump's sentiment.csv")
df3=pd.read_csv("/content/drive/MyDrive/DonaldTrump (p) 2017-2020 - Sheet1 - DonaldTrump (p) 2017-2020 - Sheet1 (1).csv")

In [4]:
df1.head()

,Unnamed: 0,date,id,link,retweet,text,author
0,0.0,Oct 7,7.846092e+17,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1.0,Oct 10,7.856088e+17,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.c...,DonaldTrump
2,2.0,Oct 8,7.848410e+17,/realDonaldTrump/status/784840992734064641,False,The media and establishment want me out of the...,DonaldTrump
3,3.0,Oct 8,7.847674e+17,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4.0,Oct 10,7.855613e+17,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #A...,DonaldTrump


In [5]:
df1.isna().sum()

Unnamed: 0    2
date          2
id            2
link          2
retweet       2
text          2
author        2
dtype: int64

In [6]:
df1.loc[df1.isnull().any(axis=1)]

,Unnamed: 0,date,id,link,retweet,text,author
17215,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17216,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df1.drop([17215,17216],inplace=True)

In [8]:
df2.head()

,user_id,freq.before presidency,freq.during pesidency
0,@realdonaldtrump,2846,2659
1,@barackobama,874,9
2,@mittromney,360,5
3,@apprenticenbc,182,1
4,@foxnews,153,534


In [9]:
df2.head(20)

,user_id,freq.before presidency,freq.during pesidency
0,@realdonaldtrump,2846,2659
1,@barackobama,874,9
2,@mittromney,360,5
3,@apprenticenbc,182,1
4,@foxnews,153,534
5,@foxandfriends,127,362
6,@cnn,102,144
7,@megynkelly,74,1
8,@ivankatrump,74,129
9,@nbc,71,4


In [10]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.0 MB/s eta 0:00:00


In [11]:
#sentiment analysis for the period before presidency
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from statistics import mode

# Create a SentimentIntensityAnalyzer object
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiment label
def get_sentiment_label(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"


# Perform sentiment analysis and add sentiment column
df1['Sentiment_Scores'] = df1['text'].apply(lambda text: analyzer.polarity_scores(text))
df1['Sentiment'] = df1['Sentiment_Scores'].apply(lambda scores: get_sentiment_label(scores['compound']))

# Assuming you have two dataframes df1 and df2
# Iterate through df2
for index, row in df2.iterrows():
    userid = row['user_id']

    # Filter df1 to get tweets containing the userid
    matching_tweets = df1[df1['text'].str.contains(userid, case=False)]

    if not matching_tweets.empty:
        # Calculate sentiment scores for matching tweets
        matching_tweets['sentiment_score'] = matching_tweets['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

        # Get sentiment labels using the provided function
        matching_tweets['sentiment_label'] = matching_tweets['sentiment_score'].apply(get_sentiment_label)

        # Calculate the mode sentiment
        mode_sentiment = mode(matching_tweets['sentiment_label'])

        # Add mode sentiment to df2
        df2.at[index, 'sentiment_before_presidency'] = mode_sentiment

# Save the updated df2 with sentiment information
df2.to_csv('df2_with_sentiment.csv', index=False)


<ipython-input-11-3aa4584f6fe4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_tweets['sentiment_score'] = matching_tweets['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
<ipython-input-11-3aa4584f6fe4>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_tweets['sentiment_label'] = matching_tweets['sentiment_score'].apply(get_sentiment_label)
<ipython-input-11-3aa4584f6fe4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [12]:
df2.head()

,user_id,freq.before presidency,freq.during pesidency,sentiment_before_presidency
0,@realdonaldtrump,2846,2659,Positive
1,@barackobama,874,9,Negative
2,@mittromney,360,5,Positive
3,@apprenticenbc,182,1,Positive
4,@foxnews,153,534,Positive


In [13]:
df3.head()

,id,text,device,retweets,fa,date
0,9.480000e+17,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...,Twitter for iPhone,166739,32589,12/31/2017 23:43
1,9.480000e+17,As our Country rapidly grows stronger and smar...,Twitter for iPhone,169966,36017,12/31/2017 22:18
2,9.480000e+17,"Iran, the Number One State of Sponsored Terror...",Twitter for iPhone,93121,25822,12/31/2017 22:00
3,9.480000e+17,"What a year it’s been, and we're just getting ...",Twitter Media Studio,117478,26486,12/31/2017 19:06
4,9.480000e+17,My deepest condolences to the victims of the t...,Twitter for iPhone,75389,14746,12/31/2017 18:36


In [14]:
#sentiment analysis for the period during presidency

# Perform sentiment analysis and add sentiment column
df3['Sentiment_Scores'] = df3['text'].apply(lambda text: analyzer.polarity_scores(text))
df3['Sentiment'] = df3['Sentiment_Scores'].apply(lambda scores: get_sentiment_label(scores['compound']))

for index, row in df2.iterrows():
    userid = row['user_id']

    # Filter df3 to get tweets containing the userid
    matching_tweets = df3[df3['text'].str.contains(userid, case=False)]

    if not matching_tweets.empty:
        # Calculate sentiment scores for matching tweets
        matching_tweets['sentiment_score'] = matching_tweets['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

        # Get sentiment labels using the provided function
        matching_tweets['sentiment_label'] = matching_tweets['sentiment_score'].apply(get_sentiment_label)

        # Calculate the mode sentiment
        mode_sentiment = mode(matching_tweets['sentiment_label'])

        # Add mode sentiment to df2
        df2.at[index, 'sentiment_after_presidency'] = mode_sentiment

# Save the updated df2 with sentiment information
df2.to_csv('df2_with_sentiment.csv', index=False)


<ipython-input-14-8ad7cf0aa5be>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_tweets['sentiment_score'] = matching_tweets['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
<ipython-input-14-8ad7cf0aa5be>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_tweets['sentiment_label'] = matching_tweets['sentiment_score'].apply(get_sentiment_label)
<ipython-input-14-8ad7cf0aa5be>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [15]:
df2.head()

,user_id,freq.before presidency,freq.during pesidency,sentiment_before_presidency,sentiment_after_presidency
0,@realdonaldtrump,2846,2659,Positive,Positive
1,@barackobama,874,9,Negative,Positive
2,@mittromney,360,5,Positive,Positive
3,@apprenticenbc,182,1,Positive,Neutral
4,@foxnews,153,534,Positive,Positive


In [16]:
csv_file_path = 'sentimental_analysis_Trump.csv'
df2.to_csv(csv_file_path, index=False)

In [17]:
df1.head()

,Unnamed: 0,date,id,link,retweet,text,author,Sentiment_Scores,Sentiment
0,0.0,Oct 7,7.846092e+17,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral
1,1.0,Oct 10,7.856088e+17,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.c...,DonaldTrump,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",Neutral
2,2.0,Oct 8,7.848410e+17,/realDonaldTrump/status/784840992734064641,False,The media and establishment want me out of the...,DonaldTrump,"{'neg': 0.207, 'neu': 0.688, 'pos': 0.105, 'co...",Negative
3,3.0,Oct 8,7.847674e+17,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump,"{'neg': 0.0, 'neu': 0.481, 'pos': 0.519, 'comp...",Positive
4,4.0,Oct 10,7.855613e+17,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #A...,DonaldTrump,"{'neg': 0.0, 'neu': 0.504, 'pos': 0.496, 'comp...",Positive


In [18]:
df2.head()

,user_id,freq.before presidency,freq.during pesidency,sentiment_before_presidency,sentiment_after_presidency
0,@realdonaldtrump,2846,2659,Positive,Positive
1,@barackobama,874,9,Negative,Positive
2,@mittromney,360,5,Positive,Positive
3,@apprenticenbc,182,1,Positive,Neutral
4,@foxnews,153,534,Positive,Positive


In [19]:
df3.head()

,id,text,device,retweets,fa,date,Sentiment_Scores,Sentiment
0,9.480000e+17,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...,Twitter for iPhone,166739,32589,12/31/2017 23:43,"{'neg': 0.0, 'neu': 0.587, 'pos': 0.413, 'comp...",Positive
1,9.480000e+17,As our Country rapidly grows stronger and smar...,Twitter for iPhone,169966,36017,12/31/2017 22:18,"{'neg': 0.202, 'neu': 0.411, 'pos': 0.387, 'co...",Positive
2,9.480000e+17,"Iran, the Number One State of Sponsored Terror...",Twitter for iPhone,93121,25822,12/31/2017 22:00,"{'neg': 0.162, 'neu': 0.668, 'pos': 0.17, 'com...",Negative
3,9.480000e+17,"What a year it’s been, and we're just getting ...",Twitter Media Studio,117478,26486,12/31/2017 19:06,"{'neg': 0.0, 'neu': 0.669, 'pos': 0.331, 'comp...",Positive
4,9.480000e+17,My deepest condolences to the victims of the t...,Twitter for iPhone,75389,14746,12/31/2017 18:36,"{'neg': 0.136, 'neu': 0.628, 'pos': 0.236, 'co...",Positive
